# Analysis example

这个 notebook 是一个占位文件，用来展示如何加载 `read01.txt`，运行解析器，并可视化可能的 MRS/EEG 指标（需要后续填充真实数据和分析代码）。

In [ ]:
# 安装依赖（如未安装，请在终端运行）
# pip install numpy matplotlib scipy pandas

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

plt.rcParams['figure.figsize'] = (10, 4)


In [ ]:
# 读取 MRS 示例 CSV
from pathlib import Path
root = Path(__file__).resolve().parents[2] if '__file__' in globals() else Path('e:/tools/begin/AstroCore')
df = pd.read_csv(root / 'data' / 'mrs_sample.csv')
df.head()


In [ ]:
# 绘制 MRS 指标随时间变化（不同脑区）
fig, axes = plt.subplots(1, 3, figsize=(15,4), sharex=True)
for i, col in enumerate(['glucose', 'lactate', 'ketone']):
    for region, g in df.groupby('region'):
        axes[i].plot(g['time_min'], g[col], marker='o', label=region)
    axes[i].set_title(col)
    axes[i].set_xlabel('time (min)')
    axes[i].legend()
plt.tight_layout()
plt.show()


## 模拟 EEG 信号（时域与功率谱）

接下来我们生成一个 10 秒钟、采样率 1000 Hz 的模拟 EEG：基线为 1/f 颜色噪声，并在中间时段注入 40 Hz 的伽马增强成分。


In [ ]:
fs = 1000  # sampling rate
t = np.arange(0, 10, 1/fs)
# 生成 1/f 噪声（简单方法)
def pink_noise(n):
    # generate white noise and filter in frequency domain to approximate 1/f
    wn = np.random.randn(n)
    f = np.fft.rfftfreq(n, 1/fs)
    S = np.where(f==0, 0, 1/np.sqrt(f))
    W = np.fft.rfft(wn) * S
    return np.fft.irfft(W, n)

eeg = pink_noise(len(t)) * 20e-6  # scale to microvolts-ish
# 在 3-7 秒之间注入 40Hz 成分
mask = (t >= 3) & (t <= 7)
eeg[mask] += 5e-6 * np.sin(2 * np.pi * 40 * t[mask])

# 绘制时域片段
plt.figure(figsize=(12,3))
plt.plot(t[0:2000], eeg[0:2000])
plt.title('Simulated EEG (first 2 seconds)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (V)')
plt.show()

# 计算并绘制功率谱密度（Welch）
f, Pxx = signal.welch(eeg, fs=fs, nperseg=2048)
plt.figure(figsize=(8,4))
plt.semilogy(f, Pxx)
plt.xlim(0,100)
plt.title('EEG Power Spectral Density (Welch)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD')
plt.show()


## 如何替换为真实数据

- 若有真实 MRS CSV，请用相同列名（region,time_min,glucose,lactate,ketone）替换 `data/mrs_sample.csv`。
- 若有真实 EEG（例如 EDF 或 FIF），建议使用 MNE-Python 载入并代替本 notebook 的模拟段（我可以帮你编写加载示例）。
